# 针对录取情况数据进行预测
# 采用一层神经网络
# 采用标准梯度下降法处理


In [48]:
#
import numpy as np
import pandas as pd

admissions = pd.read_csv(r'C:\Users\zhangwenqi\Desktop\神经网络test\第一课神经网络\第三章 实现梯度下降\binary.csv')

# Make dummy variables for rank  进行热独编码，将rank排名进行处理
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features #标准化处理，将gpa和gre处理为均值为0，方差为1的数据
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(42)   #随机分割样本为测试和训练
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets  将输入和标签分开
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

c:\users\zhangwenqi\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [31]:
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))


In [49]:
n_records, n_features = features.shape#360 ，6
last_loss = None
weights = np.random.normal(scale=1 / n_features**.5, size=n_features) #6个权重    
#这里将方差设置为1/6，是为了避免出现太大的权重初值，保证输出到激活函数的值，均在0值的附近，避免太大导致梯度下降困难

In [36]:
weights

array([-0.29912034, -0.23608056, -0.05540639, -0.32394707, -0.15041908,
        0.06417739])

In [50]:
epochs = 1000
learnrate = 0.1

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output

        # TODO: Calculate the output
        output =  sigmoid(np.dot(x, weights))#x是6*1  weights是1*6

        # TODO: Calculate the error
        error = y - output

        # TODO: Calculate the error term
        error_term = error * output * (1 - output)

        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w +=  error_term * x

    # TODO: Update weights using the learning rate and the average change in weights
    weights +=learnrate * del_w / n_records  #每次迭代中计算完360组数据后得到平均的权重调整值，然后再去调整

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))


Train loss:  0.25566110368658035
Train loss:  0.23591172518815276
Train loss:  0.22518772654417624
Train loss:  0.21815817681052682
Train loss:  0.21321866101474357
Train loss:  0.2096454272294604
Train loss:  0.20701012325358015
Train loss:  0.2050333003153721
Train loss:  0.20352672570450478
Train loss:  0.20236142914307037
Prediction accuracy: 0.750


In [39]:
predictions

array([False, False, False, False, False, False, False, False, False,
       False,  True, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False])